In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [72]:
text = 'philo.pdf'

In [73]:
pdfloader=PyPDFLoader(text)
doc=pdfloader.load()

In [74]:
doc=doc[5:71]


In [75]:
text = ""

for page in doc:
    text += page.page_content
    
text = text.replace('\t', ' ')
#text = text.replace('\u202f', ' ')

In [ ]:
print(text)

In [77]:
from dotenv import load_dotenv
load_dotenv()

True

In [78]:
from langchain.llms import HuggingFaceHub
llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", model_kwargs={"temperature":0.1,'max_new_tokens':4000})

e:\multiplellm\multiplellm\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [79]:
#from langchain.llms import CTransformers
#llm=CTransformers(model='model/llama-2-7b-chat.ggmlv3.q8_0.bin',
                      #model_type='llama',
                      #config={'temperature':0,'context_length': 4096}
                      #)



In [80]:
llm.get_num_tokens(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (31897 > 1024). Running this sequence through the model will result in indexing errors


31897

In [81]:
text_splitter = RecursiveCharacterTextSplitter(
                                                separators=['\n\n', '\n', '(?=>\. )', ' ', ''], 
                                               chunk_size=1000, 
                                               chunk_overlap=500
                                               )

In [82]:
docs = text_splitter.create_documents([text])

In [83]:
num_docs = len(docs)

In [84]:
num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

In [85]:
print (f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

Now we have 170 documents and the first one has 349 tokens


In [88]:
import chromadb
import os
PATH_DB=os.path.join(os.getcwd(),"save_vectorstore_philo")


In [89]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

#embeddings= HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
embeddings=HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
vectorstore= Chroma.from_documents(documents=docs,embedding=embeddings,persist_directory=PATH_DB)

load INSTRUCTOR_Transformer
max_seq_length  512


In [90]:
vectorstore.persist()

In [18]:
#pour load un vectorstore
#from langchain.vectorstores import Chroma
#import os
#PATH_DB=os.path.join(os.getcwd(),"save_vectorstore")
#vectordb222 = Chroma(persist_directory=PATH_DB)

In [121]:
vectorstore.get()

In [92]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
retireval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=vectorstore.as_retriever())
chain=load_qa_chain(llm,chain_type="stuff")

In [117]:
#query=" donne les points clés à retenir du document? il faudra faire abstraction des mentions légales et la réponse doit être formulé par un spécialiste du bâtiment et rédigé en français "
#query="dresse un bilan de la situation en angleterre décrit dans le document, la réponse doit être ilustré avec au moins 10 pourcentages et formulé par un spécialiste du bâtiment et rédigé en français"
query="tu es un grand philosophe et tu expliciteras les axes de réflexions concernant la thématique entre l'histoire et la liberté? réponse en francais  "
matching_results=vectorstore.similarity_search(query,k=10)

In [118]:
response=chain.run(input_documents=matching_results,question=query)

In [119]:
print(response)

 Certainement, je vais vous présenter les axes de réflexion concernant la thématique entre l'histoire et la liberté.

1. L'histoire comme facteur de la libération de l'homme:
- Histoire, source d'instruction: L'histoire peut être un outil d'apprentissage et d'éducation, en nous permettant de comprendre les événements passés et de tirer des leçons pour l'avenir.
- Histoire, source d'inspiration: L'histoire peut également être une source d'inspiration, en nous montrant des exemples de personnes qui ont lutté pour leur liberté et qui ont réussi à surmonter les obstacles.

2. L'histoire comme facteur d'aliénation:
- Histoire, source de nostalgie: L'histoire peut être une source de nostalgie, en nous faisant regretter un passé idéalisé et en nous empêchant de nous concentrer sur le présent et l'avenir.
- Histoire, source de traumatisme: L'histoire peut également être une source de traumatisme, en nous rappelant des événements douloureux et en nous empêchant de tourner la page.

3. Philosoph

In [71]:
result = retireval_chain({"query": query})
result["result"]

' The hypotheses of transformation of the surfaces of the parc of bureaux franciliens are between 0.6% and 1.3% of the total of the surfaces of the parc of bureaux franciliens.'

In [17]:
#summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce',
                                      #verbose=True # Set verbose=True if you want to see the prompts being used
                                    #)

In [21]:
#output = summary_chain.run(input_documents=docs,token_max=4096)

In [ ]:
#print(output)

In [ ]:
#n=150
#[output[i:i+n] for i in range(0, len(output), n)]

In [126]:
from langchain.prompts import PromptTemplate
map_prompt =  "[INST] tu es un philosophe. Ecrit un résumé précis en francais de :{text}Résumé précis: [/INST]"
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])


In [127]:
combine_prompt = "[INST] tu es un philosophe. Ecrit un résumé en francais en 200 mots de :{text}Résumé précis: [/INST]"
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [128]:
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm,
                                     chain_type='map_reduce',
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template,
#                                      verbose=True
                                    )

In [129]:

#summary_chain = load_summarize_chain(llm=llm,
                                     #chain_type='refine',
                                     #refine_prompt=map_prompt_template,
                                     
#                                      verbose=True
                                    #)

In [130]:
output = summary_chain.run(input_documents=docs)

In [131]:
print(output)

 La dissertation philosophique et le commentaire philosophique sont deux exercices qui permettent de développer des compétences d'argumentation, d'analyse et de réflexion critique. La dissertation consiste à défendre une thèse sur un sujet donné, en utilisant des arguments logiques et rationnels, tandis que le commentaire vise à analyser et interpréter un texte philosophique.

Ces exercices permettent de développer des compétences utiles dans divers aspects de la vie, en particulier la capacité à argumenter de manière claire et convaincante. La philosophie aborde des questions centrales telles que la question de l'universalité de la morale ou l'existence de Dieu, qui soulèvent des enjeux théoriques et pratiques considérables.

La lecture attentive et structurée des textes philosophiques est essentielle pour en comprendre la logique argumentative. Des exemples de philosophes et de leurs contributions importantes à des questions philosophiques centrales sont présentés, tels que John Lock